In [1]:
!pip install pandasql
!pip install pgeocode
import pandas as pd
import numpy as np
import pandasql as sql
import pgeocode

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26817 sha256=ab1389c17e8a3d4fb4089334e965cd4e8b1614e15f6811f833be80d6e206e5bf
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [5]:
#get postal data from wikipedia
df1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df2=sql.sqldf("select *from df1 where Borough !='Not assigned'")

#M5A arbourfront and Regent Park are in 1 row by default

#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
df2['Neighbourhood'] = np.where(df2['Neighbourhood']=='Not assigned', df2['Borough'], df2['Neighbourhood'])
df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
df2.shape

(103, 3)

In [7]:
df2['longitude']=df2['latitude']=''

LatLongFinder = pgeocode.Nominatim('CA')
for index, row in df2.iterrows():
    lat_lng_coords=LatLongFinder.query_postal_code(row['Postal Code'])
    row['latitude']  = lat_lng_coords.latitude
    row['longitude'] = lat_lng_coords.longitude                                                    
df2

#there is a slight difference in longtitude and latitude because of using pgeocode instead of geocoder package

,Postal Code,Borough,Neighbourhood,longitude,latitude
0,M3A,North York,Parkwoods,-79.33,43.7545
1,M4A,North York,Victoria Village,-79.3148,43.7276
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.3626,43.6555
3,M6A,North York,"Lawrence Manor, Lawrence Heights",-79.4504,43.7223
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-79.3889,43.6641
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",-79.5076,43.6518
99,M4Y,Downtown Toronto,Church and Wellesley,-79.383,43.6656
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",-79.2505,43.7804
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",-79.4939,43.6325
